In [1]:
import core
import pandas as pd
from datetime import datetime
import ast

In [2]:
current_date = "2025-07-23"
datetime_obj = pd.to_datetime(current_date)

In [3]:
core.MonteCarloSim(match_id=4640)

Simulating Ceará vs Mirassol
   minutes_played                date  days_ago
0               0 2025-04-05 18:30:00       108
1              76 2025-04-12 16:00:00       101
2              71 2025-04-15 21:30:00        98
3              44 2025-04-21 20:00:00        92
4              90 2025-05-17 16:00:00        66
5               4 2025-06-01 18:30:00        51
6              90 2025-07-13 20:30:00         9
7              70 2025-07-16 19:30:00         6
8              22 2025-07-20 11:00:00         2
   minutes_played                date  days_ago
0              90 2025-04-05 18:30:00       108
1              90 2025-04-12 16:00:00       101
2              64 2025-04-15 21:30:00        98
3              90 2025-06-01 18:30:00        51
4              90 2025-06-04 20:00:00        48
5              90 2025-07-13 20:30:00         9
6              90 2025-07-16 19:30:00         6
7              90 2025-07-20 11:00:00         2
   minutes_played                date  days_ago
0          